# ENGN2350 Data-Driven Design and Analysis of Structures and Materials

Coding exercises to explore the [`f3dasm`](https://f3dasm.readthedocs.io/en/latest/) package.

**General instructions**:

- Read the questions and answer in the cells under the "PUT YOUR CODE IN THE CELL BELOW" message.
- Work through the notebook and make sure you fill in any place that says `YOUR CODE HERE` or `YOUR ANSWER HERE`. You can remove the `'raise NotImplementedError()'` code.
- After "END OF YOUR CODE" , there is a cell that contains simple tests (with `assert` statements) to see if you did the exercises correctly. Not all exercises have tests! If you run the cell containing the tests and no error is given, you have succesfully solved the exercise!
- Make sure you have the right version of `f3dasm` (1.5.4). The cell below checks this:

- **ONLY WORK ON THE EXERCISE IN A JUPYTER NOTEBOOK ENVIRONMENT**

> The homework assignments are generated and automatically graded by the `nbgrader` extension. If you open and save the notebook in Google Colab, metadata from Colab will be added, and the `nbgrader` metadata will be altered. As a result, `nbgrader` will be unable to automatically grade your homework. Therefore, we kindly ask students to only work on the notebook in Jupyter Notebook.

- **DO NOT ADD OR REMOVE CELLS IN THE NOTEBOOK**

> Most cells containing tests are set to read-only, but VS Code can bypass this restriction. Modifying or removing cells in the notebook may disrupt the `nbgrader` system, preventing automatic grading of your homework.

**Instructions for handing in the homework**

- Upload the Jupyter Notebook (`.ipynb file`) to canvas

If there are any questions about the homework, send Martin an email (m.p.vanderschelling@tudelft.nl)

We make sure you have the right version of `f3dasm` (1.5.4). The cell below checks this:

In [ ]:
import f3dasm
import IPython

# Check if the f3dasm version is correct
assert f3dasm.__version__ == '1.5.4', "Your version of f3dasm is incorrect, please update it."

# Check if the IPython version is at least 3
assert IPython.version_info[0] >= 3, "Your version of IPython is too old, please update it."

# If no assert statements are triggered, print a succes message
print("Your environment is set up correctly :)")

You can put your name in the cell below:

In [ ]:
NAME = ""

---

## Homework 9

In this homework, you will delve into optimization using the `f3dasm` package.

By the end of this homework, you will:
- Gain a deeper understanding of the strengths and weaknesses of various optimization algorithms.
- Learn how initial conditions and hyperparameters affect optimization performance.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import f3dasm

---

### Exercise 1

`f3dasm` does not have a lot of optimization algorithms built in. In order to access more optimization algorithms, you are going to install the `f3dasm_optimize` extension library:

1.1 Install the `f3dasm_optimize` library. You can find the instructions on the [documentation page](https://bessagroup.github.io/f3dasm_optimize/)

```
pip install f3dasm_optimize
```

By installing this library, additional built-in default optimizers are integrated into the `f3dasm` framework. These optimizers, adapted from other libraries, are made compatible with the `ExperimentData` object. To use them, you'll also need to install the corresponding source libraries.

1.2 Install the `optax` and `evosax` libraries in your environment in order to get access to gradient-based and gradient-free optimizers:

```
pip install optax evosax
```

In order to check if the optimizers are succesfully loaded into `f3dasm`, you can inspect the `f3dasm.optimization.OPTIMIZERS` list:

If everything was done correctly, more optimizers like `'EvoSaxCMAES'` and`'Adam'` are added to the `OPTIMIZERS` list. [More optimizers](https://bessagroup.github.io/f3dasm_optimize/rst_doc_files/optimizers.html#) can be installed by following the instructions in the documentation.

In [ ]:
# This cell is to check if you installed f3dasm_optimize correctly
try:
    import f3dasm_optimize
except ModuleNotFoundError:
    assert False, "f3dasm_optimize is not installed!"

from f3dasm.optimization import OPTIMIZERS

assert 'EvoSaxCMAES' in OPTIMIZERS, "EvoSax optimizers are not correctly installed!"
assert 'Adam' in OPTIMIZERS, "Optax optimizers are not correctly installed!"

> If you didn't manage to install the `f3dasm_optimize` library, you can do Exercise 2 with the `'Nelder-Mead'` optimizer instead of the `'Adam'` optimizer.

---

### Exercise 2

We are going to evaluate the performance of various optimizers on various benchmark functions. In order to do this, a function is given below that constructs a benchmark function with a given dimensionality and optimizes it with the requested optimizer:

In [ ]:
from f3dasm.design import make_nd_continuous_domain
from f3dasm import ExperimentData
from typing import Optional

def optimize_benchmark_fn(benchmark_fn: str, optimizer: str, dimensionality: int, noise: float, seed: int, iterations: int,
                         hyperparameters: Optional[dict] = None) -> ExperimentData:

    # If no overwriting hyperparameters are given, create an empty dictionary
    if hyperparameters is None:
        hyperparameters = {}

    # Add at least the seed as an 'hyperparameter' (this is required to enable reproducibility)
    hyperparameters['seed'] = seed

    # Create a single-objective, continuous domain with bounds [0., 1.]
    domain = make_nd_continuous_domain([[0.,1.]]*dimensionality)

    # Create an empty ExperimentData object from the domain
    data = ExperimentData(domain=domain)

    # Set the kwargs for the data generator
    kwargs= {'scale_bounds': domain.get_bounds(), 'offset': False, 'seed': seed, 'noise': noise}

    # Optimize the benchmark function with the given optimizer
    # The x0 selection strategy is set to 'new', hence first sampling new candidate(s) as a starting point
    data.optimize(optimizer=optimizer, data_generator=benchmark_fn, kwargs=kwargs, 
                  iterations=iterations, x0_selection='new', hyperparameters=hyperparameters)

    # Return the data
    return data

The function takes the following inputs:

- **`benchmark_fn`**: The name of a [built-in benchmark function](https://f3dasm.readthedocs.io/en/latest/rst_doc_files/defaults.html#implemented-benchmark-functions) from `f3dasm`.
- **`optimizer`**: The name of a [built-in optimizer](https://bessagroup.github.io/f3dasm_optimize/rst_doc_files/optimizers.html#).
- **`dimensionality`**: The number of continuous input dimensions.
- **`noise`**: Magnitude of Gaussian noise added to the objective value. Value denotes the fraction of the objective value that will be used as the standard deviation in the Gaussian distribution.
- **`seed`**: The seed for generating initial candidate solutions (this is done with random uniform sampling).
- **`iterations`**: The total number of function evaluations allowed for optimizing the function.
- **`hyperparameters`**: _(optional)_ A dictionary to set non-default hyperparameters for the optimizer.

The function tries to minimize the benchmark function $f(\vec{x})$ with a finite budget while changing the input $\vec{x} = (x^0, x^1, x^2 \dots x^D)$ with $D$ the dimensionality of the search space. We constrain $\vec{x}$ to be in a unit domain, $[0., 1.]^D$, 

The output of the `optimize_benchmark_fn` is an `ExperimentData` object containing the entire history of function evaluations during optimization.

---

2.1 _(3 points)_ Optimize the noiseless 2D `'Sphere'` function with the `'Adam'` optimizer for $500$ iterations by using the `optimize_benchmark_fn`. Use $123$ as the random seed. 

Make a plot where you show the progression of the objective values (y-axis) w.r.t the number of function evaluations (x-axis).

---

YOUR CODE IN THE CELL BELOW

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

END OF YOUR CODE

---

In addition to tracking how the objective function value evolves with each update step, visualizing the input parameters throughout the iterations can provide valuable insights. This visualization, referred to as the optimizer's trajectory, can be achieved by plotting it over a contour plot of the benchmark function.

A **contour plot** is a two-dimensional visualization of a three-dimensional surface. In this plot, we represent constant values of the function on a 2D plane using contour lines, or "level curves." Each line on the plot connects points where the function value is the same. This allows us to easily see how the function changes over a domain, highlighting areas of increase and decrease.

In optimization, contour plots are especially useful for understanding the shape of the function landscape, which can reveal minima, maxima, and saddle points.

We can generate a contour plot of benchmark functions using the following function:

In [ ]:
from itertools import product

def plot_function(data_generator: str, resolution: int = 31):
    # Step 1: Create the input data by creating a grid of points, equally spaced
    x0 = np.linspace(0., 1., resolution) # Create equally spaced points between 0 and 1, x0
    x1 = np.linspace(0., 1., resolution) # Create equally spaced points between 0 and 1, x1
    input_data = np.array(list(product(x0, x1))) # Create a grid of points

    # Step 2: Create the experiment data
    domain = make_nd_continuous_domain([[0.,1.]]*2) # Create a 2D domain
    experiment_data = ExperimentData(input_data=input_data, domain=domain)

    # Step 3: Evaluate the grid of points on the data generator
    experiment_data.evaluate(data_generator=data_generator, kwargs={
        'scale_bounds': domain.get_bounds(), 'offset': False})

    # Step 3.1: Retrieve the best found point in the grid, estimating the 'global minimum'
    x_min, _ = experiment_data.get_n_best_output(1).to_numpy()  
    
    # Step 4: Create the contour plot
    array_in, array_out = experiment_data.to_numpy()
    X = array_in[:, 0].reshape(resolution, resolution) # Reshape in order to make the contour plot
    Y = array_in[:, 1].reshape(resolution, resolution) # Reshape in order to make the contour plot
    Z = array_out.reshape(resolution, resolution)
    fig, ax = plt.subplots(figsize=(8, 6))
    contour = ax.contour(X, Y, Z, levels=50, cmap='viridis', zorder=0)
    ax.contourf(X, Y, Z, levels=50, cmap='viridis', alpha=0.7, zorder=0)
    ax.scatter(x_min[:, 0], x_min[:, 1], color='k', marker='x', label='global minimum') # Display the global minimum estimate with a black cross
    fig.colorbar(contour)
    ax.set_xlabel("$x_0$")
    ax.set_ylabel("$x_1$")
    return fig, ax

The function takes the following inputs:

- **`benchmark_fn`**: The name of a [built-in benchmark function](https://f3dasm.readthedocs.io/en/latest/rst_doc_files/defaults.html#implemented-benchmark-functions) from `f3dasm`.
- **`resolution`**: _(optional)_ The number of equally spaced points per dimension used for constructing the contour plot, by default $31$.

The function returns a Matplotlib figure (`plt.Figure`) and axis (`plt.Axes`) object, allowing you to use the axis to overlay the optimization trajectory.

---

2.2 _(2 points)_ Create a contour plot of the 'Sphere' function using the `plot_function` function and plot the trajectory (= 2D input dimension) of the Adam optimizer created in the previous exercise on top of this landscape.

---

YOUR CODE IN THE CELL BELOW

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

END OF YOUR CODE

---

2.3 _(3 points)_ Optimize the same benchmark function in exercise 2.1, but now use the Adam optimizer with **4 different values of the learning rate hyperparameter**: $10^{-3}$, $10^{-2}$, $10^{-1}$ and $10^{0}$. Use the same seed $123$ and run for a maximum of $500$ function evaluations.
- Create a contour plot with the trajectories of the four Adam optimizers
- Make one figure where you plot the objective value w.r.t. the number of function evaluations for the four runs.
---

YOUR CODE IN THE CELL BELOW

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

END OF YOUR CODE

---

2.4 Conceptual questions: 

- _(1 point)_ What does the learning rate hyperparameter control for the Adam optimizer?
---

YOUR ANSWER IN THE CELL BELOW

YOUR ANSWER HERE

END OF YOUR ANSWER

---
- _(1 point)_ What do you observe on the optimization trajectory when you change the learning rate? What do you see when we choose a learning rate that is too low or too high?
---

YOUR ANSWER IN THE CELL BELOW

YOUR ANSWER HERE

END OF YOUR ANSWER

---
- _(1 point)_ How can you get a suitable learning rate for the Adam optimizer on your optimization problem?
---

YOUR ANSWER IN THE CELL BELOW

YOUR ANSWER HERE

END OF YOUR ANSWER

---

2.5 _(2 points)_ Redo exercise 2.1 and 2.2 but know use the **gradient-free optimizer CMA-ES (Covariance Matrix Adaptation Evolution Strategy)**. Use the version implemented by the EvoSax library by using the `'ExoSaxCMAES'` built-in optimizer.

---

YOUR CODE IN THE CELL BELOW

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

END OF YOUR CODE

---

### Exercise 3

Initial conditions play a significant role in the behavior and outcome of optimization algorithms. The starting point of an optimization process can influence the trajectory taken by the optimizer and determine whether it converges to a global or local minimum.

In the `optimize_benchmark_fn` function, the initial starting point, $\vec{x}_0$, is generated by sampling from a random uniform distribution. By using different `seed` values, we can alter the random number generation, creating new starting points for the optimization process.

---

3.1 _(3 points)_ Repeat exercise 2.1 but now with a few different seeds: $123$, $124$, $125$ and $126$. Create two figures:
- Plot the objective value (y-axis) against the function evaluation count (x-axis) for each outcome in one figure.*
- Plot all the trajectories in one contour plot.

_\* You might want to use a logarithmic scale on the objective value to observe the differences more clearly_

---

YOUR CODE IN THE CELL BELOW

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

END OF YOUR CODE

---

3.2 _(2 points)_ Repeat exercise 3.1 but now with a different benchmark function the `'Branin'` function (more information about this function can be found [here](https://www.sfu.ca/~ssurjano/branin.html))

---

YOUR CODE IN THE CELL BELOW

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

END OF YOUR CODE

---

3.3 Conceptual questions:

- _(1 point)_ Can you explain in words what the influence is on the outcome of optimization if you are using different initial conditions on the two functions?

---

YOUR ANSWER IN THE CELL BELOW

YOUR ANSWER HERE

END OF YOUR ANSWER

---

### Exercise 4

We will now evaluate the performance of various optimization algorithms using different benchmark functions. In this homework, we define the performance of an optimization algorithm as the **best objective value found** (cumulative minimum in the case of minimization problems) **after a specified number of function evaluations**. To account for the effect of initial conditions, we will report the **median result over multiple runs**, each with a different starting point $\vec{x}_0$.

We will consider default hyper-parameters for any given optimizer at this point

---

4.1 _(3 points)_ Complete the function `calc_performance` to calculate the performance value for a given optimizer and benchmark function.

**Requirements:**
- The function should call `optimize_benchmark_fn` multiple times using the provided arguments, each with a different `seed`.
- It should compute the cumulative minimum\* of the objective value over the number of function evaluations for each run.
- Finally, the median value across all runs should be calculated and stored in `median_performance`, which will be returned as a one-dimensional `np.ndarray`.

**Function Arguments:**
- **`benchmark_fn`**: The name of a [built-in benchmark function](https://f3dasm.readthedocs.io/en/latest/rst_doc_files/defaults.html#implemented-benchmark-functions) from `f3dasm`.
- **`optimizer`**: The name of a [built-in optimizer](https://bessagroup.github.io/f3dasm_optimize/rst_doc_files/optimizers.html#).
- **`dimensionality`**: The number of continuous input dimensions.
- **`noise`**: Magnitude of Gaussian noise added to the objective value. Value denotes the fraction of the objective value that will be used as the standard deviation in the Gaussian distribution.
- **`iterations`**: The total number of function evaluations allowed for the optimization.
- **`realizations`**: The number of runs performed with different initial conditions.
- **`seed`**: The seed for generating initial candidate solutions. Successive realizations will use seeds incremented from this base, e.g., `seed=123` with `realizations=3` results in `optimize_benchmark_fn` being called with seeds $123$, $124$, and $125$.

_\* Hint: you might want to use the `numpy.minimum.accumulate` function!_ 

---

PUT YOUR CODE IN THE CELL BELOW

In [ ]:
def calc_performance(benchmark_fn: str, optimizer: str, dimensionality: int, noise: float, iterations: int, realizations: int, 
                     seed: int) -> np.ndarray:
    seeds = np.arange(realizations) + seed
    # YOUR CODE HERE
    raise NotImplementedError()
    return median_performance

END OF YOUR CODE

---

4.2 _(2 points)_ Plot the performance values with respect to the number of function evaluations for a 10D noiseless Sphere function for the `'EvoSaxCMAES'`, `'L-BFGS-B'` and `'Adam'` optimizer in one figure. Use $123$ as the seed and run for $10$ realizations. As a maximum budget, you can put $500$ function evaluations 

---

YOUR CODE IN THE CELL BELOW

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

END OF YOUR CODE

---

4.3 _(4 points)_ Repeat 4.2, but now consider the following different benchmark functions:
- 10D Sphere function, noiseless (this is the function of exercise 4.2)
- 2D Branin function, noiseless
- 5D Rastrigin function, noiseless
- 20D Rastrigin function, noisy (`noise=0.2`)

Create a different figure for each of the 4 functions

--- 

YOUR CODE IN THE CELL BELOW

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

END OF YOUR CODE

---

4.4 _(2 points)_ Conceptual question: 

- Can you reflect on the results from question 4.3 and explain why some optimizers having different performances on different benchmark functions?
  In your answer, reflect on the influence of the input dimensionality, noisy/noiseless observations and the (absence of) local minima.
---

YOUR ANSWER IN THE CELL BELOW

YOUR ANSWER HERE

END OF YOUR ANSWER

---

End of the homework!